# Team Name : What's In a Name 
Members : Mahasweta, Pratap, Kinsuk 

GRA GRAbble Hackathon




In [1]:
# code EDA : Read the data and the industry mapping : 
import pandas as pd
import numpy as np

df_train=pd.read_csv("C:/Users/45018883/Desktop/GRABBLE/GRAbble_TrainCSV.csv")
df_var_class_map=pd.read_csv("C:/Users/45018883/Desktop/GRABBLE/GRABBLE_Vars_Mapping_WhatsInAName.csv")

# df_var_class_map.head()
df_train.head()

len(df_train)


34095

In [2]:

df_ind_dummy= pd.DataFrame({"Variable":['High_Risk_Ind', 'Low_Risk_Ind'], 
                    "Variable_Class":['High_Risk_Ind', 'Low_Risk_Ind']}) 
df_var_class_map=df_var_class_map.append(df_ind_dummy)
df_var_class_map

,Variable,Variable_Class
0,Rec_ID,Rec_ID
1,FLAG_NTB,FLAG_NTB
2,FLAG_LIMITED,FLAG_LIMITED
3,Industry,Industry
4,AUDITMETHOD_1,AUDITMETHOD
...,...,...
372,CashFlow_2_1_to_Turnover,CashFlow_2_1_to_Turnover
373,CashFlow_2_1_to_Equity,CashFlow_2_1_to_Equity
374,Target,Target
0,High_Risk_Ind,High_Risk_Ind


In [3]:
##Code block to udnerstand the datasets and to filter out few of the variable types : 

df_train.describe()
df_dtypes=pd.DataFrame({'Variable' : df_train.dtypes.index, 'Type' : df_train.dtypes.values})

df_dtypes[df_dtypes['Type']=='object']



,Variable,Type
3,Industry,object
4,AUDITMETHOD_1,object
5,AUDITMETHOD_2,object
6,AUDITMETHOD_3,object
7,AUDITMETHOD_4,object


In [4]:
## Missing Value Investigation: 
percent_missing = df_train.isnull().sum() * 100 / len(df_train)
missing_value_df = pd.DataFrame({'column_name': df_train.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df
selected_columns_df=missing_value_df[missing_value_df['percent_missing']<=10.0]
selected_col_list=selected_columns_df['column_name'].tolist()
df_train_missing10prct=df_train[selected_col_list]
df_train_missing10prct.head()

,Rec_ID,GROSS_PROFIT_1,NET_INCOME_1,NET_INTANGIBLES_1,NPBT_1,RETAINED_EARNINGS_1,SENIOR_NET_DEBT_1,TANGIBLE_NET_WORTH_1,TOTAL_ASSETS_1,TURNOVER_1,...,Debt_to_TNW_1,NetFixedAssets_to_TotalDebt_1,CashFlow_cagr,Cash_to_Total_assets_cagr,DEPRECIATION_2,Cash_to_TO_cagr,TotalDebt_to_TangibleAssets_2,TOTAL_DEBT_2,EBITDA_to_total_Debt_2,Debt_to_TNW_2
0,45627,2527734,150315,1,220303,812561,890812,812698,2430037,2527734,...,1.6745,1.4356,0.1315,-0.3279,59947.0,-0.0683,0.612134,1413004.0,0.141196,2.378829
1,45628,124057,28829,58767,28829,433195,233583,374428,690600,135810,...,0.6824,2.3872,-0.2714,-0.2771,8274.0,-0.3071,0.463334,277638.0,-0.024962,0.868512
2,45629,1577802,272950,22124,314816,551127,93093,529006,1207376,4998079,...,0.6383,1.5401,0.4282,0.1305,46963.0,0.1292,0.325746,262534.0,0.677136,0.841300
3,45630,3242324,-115120,0,-115120,837370,3006790,837391,9229945,11147635,...,4.2087,2.1024,1.5132,1.3629,702106.0,1.4299,0.358997,3244364.0,0.126552,3.708610
4,45631,540762,-695403,0,-695403,-855472,3145639,-841644,7777384,778696,...,-4.0618,2.1633,-0.1555,-0.1406,NaN,-0.3369,0.445503,3381428.0,0.058393,-4.127333


In [5]:
## Adding and deleting columns for treatment : 
df=df_train_missing10prct

df.columns
df.columns.to_series().groupby(df.dtypes).groups
num_list=df.select_dtypes(include=['floating','integer']).columns.tolist()
num_list.remove('Rec_ID')
num_list.remove('Target')


numlist_ind=num_list+['Industry']

print(len(df.columns))
print(len(num_list))



164
159


In [6]:
new_list = [
    list(set(df.columns).difference(num_list))
]
new_list



[['Industry', 'AUDITMETHOD_2', 'AUDITMETHOD_1', 'Rec_ID', 'Target']]

## Outlier Treatment , Missing Value Imputation by median and grouped by industry and finally normalization of the entire data set : 

In [7]:
##### Capping and flooring wioth 5%ile and 95%ile values of each numeric columns :

pd.options.mode.chained_assignment = None 

df.replace([-np.Inf,np.Inf], np.nan,inplace=True)
df_trn_ot= df.groupby(df.Industry.values).transform(lambda x: np.clip(x,x.quantile(0.05),x.quantile(0.95))if x.name in num_list else x)
df_trn_ot.head()


,Rec_ID,GROSS_PROFIT_1,NET_INCOME_1,NET_INTANGIBLES_1,NPBT_1,RETAINED_EARNINGS_1,SENIOR_NET_DEBT_1,TANGIBLE_NET_WORTH_1,TOTAL_ASSETS_1,TURNOVER_1,...,Debt_to_TNW_1,NetFixedAssets_to_TotalDebt_1,CashFlow_cagr,Cash_to_Total_assets_cagr,DEPRECIATION_2,Cash_to_TO_cagr,TotalDebt_to_TangibleAssets_2,TOTAL_DEBT_2,EBITDA_to_total_Debt_2,Debt_to_TNW_2
0,45627,2527734.00,150315.0,1.0,220303.00,812561.00,890812.0,812698.00,2430037.0,2527734.0,...,1.6745,1.4356,0.1315,-0.3279,59947.0,-0.0683,0.612134,1413004.0,0.141196,2.378829
1,45628,151678.25,28829.0,58767.0,28829.00,433195.00,233583.0,374428.00,690600.0,219028.0,...,0.6824,2.3872,-0.2714,-0.2771,8274.0,-0.3071,0.463334,277638.0,-0.024962,0.868512
2,45629,1577802.00,272950.0,22124.0,314816.00,551127.00,93093.0,529006.00,1207376.0,4998079.0,...,0.6383,1.5401,0.4282,0.1305,46963.0,0.1292,0.325746,262534.0,0.677136,0.841300
3,45630,1613675.00,-115120.0,0.0,-115120.00,837370.00,3006790.0,837391.00,9229945.0,4921810.0,...,4.2087,2.1024,1.5132,1.3629,315870.0,1.4299,0.358997,3244364.0,0.126552,3.708610
4,45631,540762.00,-513233.0,0.0,-506779.75,-503934.75,3145639.0,-420659.75,7777384.0,778696.0,...,-4.0618,2.1633,-0.1555,-0.1406,NaN,-0.3369,0.445503,3381428.0,0.058393,-4.127333


In [8]:
## MVI by inudstry wise median values :

df_trn_ot_mvi = df_trn_ot.fillna(df_trn_ot.groupby('Industry')[num_list].transform('median'))

df_check=df_trn_ot_mvi.isnull().sum()
df_check2=pd.DataFrame({'Variable':df_check.index, '#Missing_Values':df_check.values})


df_check2[df_check2['#Missing_Values']>0]


,Variable,#Missing_Values
69,AUDITMETHOD_2,920


In [9]:
## Normalize the entire data set :

from sklearn.preprocessing import StandardScaler
df_trn_ot_mvi[num_list] = StandardScaler().fit_transform(df_trn_ot_mvi[num_list])
df_trn_ot_mvi.head() 

,Rec_ID,GROSS_PROFIT_1,NET_INCOME_1,NET_INTANGIBLES_1,NPBT_1,RETAINED_EARNINGS_1,SENIOR_NET_DEBT_1,TANGIBLE_NET_WORTH_1,TOTAL_ASSETS_1,TURNOVER_1,...,Debt_to_TNW_1,NetFixedAssets_to_TotalDebt_1,CashFlow_cagr,Cash_to_Total_assets_cagr,DEPRECIATION_2,Cash_to_TO_cagr,TotalDebt_to_TangibleAssets_2,TOTAL_DEBT_2,EBITDA_to_total_Debt_2,Debt_to_TNW_2
0,45627,0.559886,-0.181468,-0.234757,-0.089815,-0.279504,0.402002,-0.329008,-0.326593,-0.425304,...,0.238732,-0.227097,-0.168877,-0.553528,-0.234646,-0.270510,1.166334,0.167613,-0.364093,0.576128
1,45628,-0.844914,-0.483687,-0.039994,-0.520113,-0.496134,-0.074487,-0.510366,-0.676543,-1.055168,...,-0.202256,0.091088,-0.557325,-0.494040,-0.617081,-0.541359,0.544175,-0.431377,-0.417728,-0.092682
2,45629,-0.001744,0.123610,-0.161437,0.122584,-0.428791,-0.176342,-0.446401,-0.572575,0.248658,...,-0.221858,-0.192155,0.117181,-0.016727,-0.330741,-0.046503,-0.031109,-0.439345,-0.191092,-0.104732
3,45630,0.019465,-0.841787,-0.234760,-0.843609,-0.265337,1.936081,-0.318790,1.041451,0.227850,...,1.365180,-0.004140,1.163262,1.426454,1.659456,1.428766,0.107920,1.133791,-0.368820,1.164992
4,45631,-0.614876,-1.832167,-0.234760,-1.723784,-1.031264,2.036746,-0.839376,0.749217,-0.902479,...,-2.311047,0.016223,-0.445582,-0.334194,-0.364753,-0.575159,0.469618,1.206102,-0.390821,-2.304979


## Industry grouping and dummy creations : 

In [10]:
## Indutrsy grouping and classification based on default rates : 

df_industry=df_trn_ot_mvi[['Rec_ID','Industry','Target']]
df_industry.head()

DR = pd.DataFrame(df_industry.groupby('Industry')['Target'].apply(lambda x:
                                                 100 * x.sum() / float(x.count())).reset_index())

ind_count=pd.DataFrame(df_industry.groupby('Industry')['Target'].apply(lambda x:
                                                 x.count()).reset_index())

ind_count.rename(columns={'Target':'Count'}, inplace=True)
DR.rename(columns={'Target':'Def_Rate'}, inplace=True)
ind_count

df_ind_dr =ind_count.merge(DR,on=['Industry'],how='left')
df_ind_dr
df_ind_dr.sort_values(by='Def_Rate',ascending=False)
# df_ind_dr.to_csv('C:/Users/45018883/Desktop/GRABBLE/def_rate.csv')


Ind_Class = {'Industry':  ['Agriculture - Dairy','Utilities','EduArtsHealthSocial','Retail','Wholesale','Manufacture','Services_SupportAdmin','Services_ProfessionalSciTech','Construction','Hospitality','RealEstate','ZZ','InformationCommunication','Mining','Transport','Financial','Agriculture - Other'],
        'Industry_Class': ['High_Risk_Ind','High_Risk_Ind','High_Risk_Ind','High_Risk_Ind','High_Risk_Ind','High_Risk_Ind','High_Risk_Ind','High_Risk_Ind','Med_Risk_Ind','Med_Risk_Ind','Med_Risk_Ind','Med_Risk_Ind','Med_Risk_Ind','Low_Risk_Ind','Low_Risk_Ind','Low_Risk_Ind','Low_Risk_Ind']}

df_industry_class = pd.DataFrame (Ind_Class, columns = ['Industry','Industry_Class'])

##Merge with the main data set : 
df_trn_ot_mvi =df_trn_ot_mvi.merge(df_industry_class,on=['Industry'],how='left')
df_trn_ot_mvi.head()




,Rec_ID,GROSS_PROFIT_1,NET_INCOME_1,NET_INTANGIBLES_1,NPBT_1,RETAINED_EARNINGS_1,SENIOR_NET_DEBT_1,TANGIBLE_NET_WORTH_1,TOTAL_ASSETS_1,TURNOVER_1,...,NetFixedAssets_to_TotalDebt_1,CashFlow_cagr,Cash_to_Total_assets_cagr,DEPRECIATION_2,Cash_to_TO_cagr,TotalDebt_to_TangibleAssets_2,TOTAL_DEBT_2,EBITDA_to_total_Debt_2,Debt_to_TNW_2,Industry_Class
0,45627,0.559886,-0.181468,-0.234757,-0.089815,-0.279504,0.402002,-0.329008,-0.326593,-0.425304,...,-0.227097,-0.168877,-0.553528,-0.234646,-0.270510,1.166334,0.167613,-0.364093,0.576128,High_Risk_Ind
1,45628,-0.844914,-0.483687,-0.039994,-0.520113,-0.496134,-0.074487,-0.510366,-0.676543,-1.055168,...,0.091088,-0.557325,-0.494040,-0.617081,-0.541359,0.544175,-0.431377,-0.417728,-0.092682,High_Risk_Ind
2,45629,-0.001744,0.123610,-0.161437,0.122584,-0.428791,-0.176342,-0.446401,-0.572575,0.248658,...,-0.192155,0.117181,-0.016727,-0.330741,-0.046503,-0.031109,-0.439345,-0.191092,-0.104732,High_Risk_Ind
3,45630,0.019465,-0.841787,-0.234760,-0.843609,-0.265337,1.936081,-0.318790,1.041451,0.227850,...,-0.004140,1.163262,1.426454,1.659456,1.428766,0.107920,1.133791,-0.368820,1.164992,Low_Risk_Ind
4,45631,-0.614876,-1.832167,-0.234760,-1.723784,-1.031264,2.036746,-0.839376,0.749217,-0.902479,...,0.016223,-0.445582,-0.334194,-0.364753,-0.575159,0.469618,1.206102,-0.390821,-2.304979,High_Risk_Ind


In [11]:
##Create industry class dummies and merge to the original data set :
ind_dummies=pd.get_dummies(df_trn_ot_mvi['Industry_Class'])
ind_dummies=ind_dummies.drop('Med_Risk_Ind',axis=1)

df_trn_ot_mvi = pd.concat([df_trn_ot_mvi, ind_dummies], axis=1)
df_trn_ot_mvi.head()


numlist_ind_dummy=num_list+['High_Risk_Ind' ,'Low_Risk_Ind' ]
df_trn_ot_mvi.head()

,Rec_ID,GROSS_PROFIT_1,NET_INCOME_1,NET_INTANGIBLES_1,NPBT_1,RETAINED_EARNINGS_1,SENIOR_NET_DEBT_1,TANGIBLE_NET_WORTH_1,TOTAL_ASSETS_1,TURNOVER_1,...,Cash_to_Total_assets_cagr,DEPRECIATION_2,Cash_to_TO_cagr,TotalDebt_to_TangibleAssets_2,TOTAL_DEBT_2,EBITDA_to_total_Debt_2,Debt_to_TNW_2,Industry_Class,High_Risk_Ind,Low_Risk_Ind
0,45627,0.559886,-0.181468,-0.234757,-0.089815,-0.279504,0.402002,-0.329008,-0.326593,-0.425304,...,-0.553528,-0.234646,-0.270510,1.166334,0.167613,-0.364093,0.576128,High_Risk_Ind,1,0
1,45628,-0.844914,-0.483687,-0.039994,-0.520113,-0.496134,-0.074487,-0.510366,-0.676543,-1.055168,...,-0.494040,-0.617081,-0.541359,0.544175,-0.431377,-0.417728,-0.092682,High_Risk_Ind,1,0
2,45629,-0.001744,0.123610,-0.161437,0.122584,-0.428791,-0.176342,-0.446401,-0.572575,0.248658,...,-0.016727,-0.330741,-0.046503,-0.031109,-0.439345,-0.191092,-0.104732,High_Risk_Ind,1,0
3,45630,0.019465,-0.841787,-0.234760,-0.843609,-0.265337,1.936081,-0.318790,1.041451,0.227850,...,1.426454,1.659456,1.428766,0.107920,1.133791,-0.368820,1.164992,Low_Risk_Ind,0,1
4,45631,-0.614876,-1.832167,-0.234760,-1.723784,-1.031264,2.036746,-0.839376,0.749217,-0.902479,...,-0.334194,-0.364753,-0.575159,0.469618,1.206102,-0.390821,-2.304979,High_Risk_Ind,1,0


## Splitting the data set into training and testing itnernally 

In [12]:
##Splitting the data set into training vs testing first : 
import sklearn.model_selection as model_selection
X=df_trn_ot_mvi.copy()
y=df_trn_ot_mvi['Target'].copy()
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80,test_size=0.20, random_state=101)
print("Length of X_Train : %d" %len(X_train))
print("Length of X_Test : %d" %len(X_test))
print("\t")
print("Default in X_Train: %d" %X_train['Target'].sum())
print("Default in X_Test: %d" %X_test['Target'].sum())
print("\t")
print("Default Rate in X_Train: %f" %(X_train['Target'].sum()*100/len(X_train)))
print("Default Rate in X_Train: %f" %(X_test['Target'].sum()*100/len(X_test)))

Length of X_Train : 27276
Length of X_Test : 6819
	
Default in X_Train: 664
Default in X_Test: 185
	
Default Rate in X_Train: 2.434375
Default Rate in X_Train: 2.713008


In [13]:
### Feature selection stage -1 :  
##Variable wise Accuracy alculation one by one under loop for all variablese using logistic regression:


from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


y_train_raw= y_train.astype(int).values
y_train_raw=np.ravel(y_train)
AR_Df=  pd.DataFrame(columns=['Variable', 'Accuracy_Score','Recall_Score','Gini'])

for i in range(len(numlist_ind_dummy)):
    Accu_Rat=[]
    var_name=[]
    x_train=[]
    ypred_train=[]
    clf=[]
    
    x_train_raw = X_train[numlist_ind_dummy[i]].values.reshape(-1, 1)
    var_name=numlist_ind_dummy[i]
    
    clf = LogisticRegression(solver='liblinear',class_weight="balanced")
    clf.fit(x_train_raw,y_train_raw)
    ypred_train_raw = clf.predict(x_train_raw)
    Accu_Score =accuracy_score(y_train_raw,ypred_train_raw)
    rec_score=recall_score(y_train_raw,ypred_train_raw)
    auc = roc_auc_score(y_train_raw,ypred_train_raw)
    gini = (2*auc -1)
    AR_Df=AR_Df.append(pd.DataFrame({'Variable' : var_name , 'Accuracy_Score' : Accu_Score ,'Recall_Score' : rec_score , 'Gini' : gini},index=[0]),ignore_index=True)


AR_Df_2= pd.merge(AR_Df,df_var_class_map, on='Variable', how='left')
AR_Df_2


,Variable,Accuracy_Score,Recall_Score,Gini,Variable_Class
0,GROSS_PROFIT_1,0.361857,0.637048,-0.007962,GROSS_PROFIT
1,NET_INCOME_1,0.422019,0.670181,0.086008,NET_INCOME
2,NET_INTANGIBLES_1,0.853094,0.171687,0.041783,NET_INTANGIBLES
3,NPBT_1,0.418573,0.692771,0.104503,NPBT
4,RETAINED_EARNINGS_1,0.391993,0.727410,0.111034,RETAINED_EARNINGS
...,...,...,...,...,...
156,TOTAL_DEBT_2,0.277827,0.674699,-0.057377,TOTAL_DEBT
157,EBITDA_to_total_Debt_2,0.288532,0.822289,0.097503,EBITDA_to_total_Debt
158,Debt_to_TNW_2,0.718177,0.350904,0.078245,Debt_to_TNW
159,High_Risk_Ind,0.392360,0.769578,0.152526,High_Risk_Ind


In [14]:
### Feature selection stage -2 :  

#Select top variables of each class : 
AR_Df_3 = AR_Df_2.set_index(['Variable']).groupby('Variable_Class')['Gini'].nlargest(1).reset_index() 
AR_Df_3


final_var_list=AR_Df_3['Variable'].tolist()
x_train_final= X_train[final_var_list]

print("# of rows in x_train_final : %d" %len(x_train_final.axes[0]))
print("# of columns in x_train_final : %d" %len(x_train_final.axes[1]))


# of rows in x_train_final : 27276
# of columns in x_train_final : 110


In [81]:
## VIF Calculation and setting threshhold at 4 and eliminating variables one after another untill max VIF becomes less than the threshol


from statsmodels.stats.outliers_influence import variance_inflation_factor   
X= x_train_final.copy()

def calculate_vif_(X, thresh=4.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) >= thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc) + 
                 '\' with VIF_Value:' + str(max(vif)))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return pd.DataFrame(X.iloc[:, variables])

df_vif_checked = calculate_vif_(X, thresh=4.0)
non_cor_vars_list=df_vif_checked.columns.tolist()

dropping 'Total_Liab_to_Total_Assets_1' at index: 103' with VIF_Value:1139.6455745248231
dropping 'Cash_growth_2_1' at index: 8' with VIF_Value:371.12905382745873
dropping 'RE_to_Assets_2_1' at index: 71' with VIF_Value:143.59019225793537
dropping 'NPBT_1' at index: 55' with VIF_Value:91.71039973231719
dropping 'RE_to_Assets_cagr' at index: 70' with VIF_Value:79.38888806825074
dropping 'Cash_to_TO_2_1' at index: 9' with VIF_Value:76.22086685556239
dropping 'RE_to_Assets_1' at index: 68' with VIF_Value:72.38999525661976
dropping 'Total_Liab_to_Total_Assets_cagr' at index: 98' with VIF_Value:62.55602234116488
dropping 'NPBT_to_Sales_1' at index: 54' with VIF_Value:61.05774141780225
dropping 'EQUITY_1' at index: 36' with VIF_Value:52.26032884284987
dropping 'NPBT_to_Sales_2_1' at index: 53' with VIF_Value:51.63754287518076
dropping 'Total_Liab_to_Total_Assets_2_1' at index: 94' with VIF_Value:44.262366525527725
dropping 'EBIT_1' at index: 21' with VIF_Value:44.07848622632748
dropping 'NPB

In [15]:

final_vif_passed_list=['CASH_AND_CASH_EQUIVALENTS_floored2', 'CURRENT_LIABILITIES_2',
       'CashFlow_2_1', 'CashFlow_2_1_to_Equity', 'CashFlow_2_1_to_Turnover',
       'Cash_to_TO_1', 'Cash_to_TO_cagr', 'Cash_to_Total_assets_1',
       'Cash_to_Total_assets_2_1', 'Curr_to_Total_Assets_2_1',
       'Curr_to_Total_Assets_cagr', 'Current_Ratio_1', 'Current_Ratio_2_1',
       'Current_Ratio_cagr', 'DEPRECIATION_2', 'Debt_to_TNW_1',
       'EBITDA_to_Tangible_Assets_2', 'EBITDA_to_Tangible_Assets_cagr',
       'EBITDA_to_curr_liab_2', 'EBITDA_to_curr_liab_2_1',
       'EBITDA_to_total_Debt_2', 'EBIT_to_TO_cagr', 'Equity_to_Assets_cagr',
       'Equity_to_Liability_2', 'Equity_to_Liability_2_1', 'FLAG_LIMITED',
       'FLAG_NTB', 'GROSS_PROFIT_1', 'High_Risk_Ind', 'Low_Risk_Ind',
       'NET_FIXED_ASSETS_2', 'NET_INCOME_1', 'NET_INTANGIBLES_1',
       'NPBT_to_TNW_1', 'NPBT_to_TNW_2_1', 'NPBT_to_TNW_cagr',
       'NetFixedAssets_to_TotalDebt_1', 'Net_Income_Margin_1',
       'Net_Income_Margin_2_1', 'Net_Income_Margin_cagr',
       'RETAINED_EARNINGS_1', 'RE_to_Equity_1', 'RE_to_Equity_2_1',
       'RE_to_Equity_cagr', 'RE_to_Tangible_Assets_2_1',
       'RE_to_Tangible_Assets_cagr', 'ROCE_2', 'ROCE_2_1', 'ROCE_cagr',
       'SENIOR_NET_DEBT_1', 'TOTAL_ASSETS_2_1', 'TO_to_Assets_1',
       'TO_to_Assets_2_1', 'TO_to_Assets_cagr',
       'TotalDebt_to_TangibleAssets_2', 'TotalLiab_to_TNW_2',
       'TotalLiab_to_TNW_2_1', 'TotalLiab_to_TNW_cagr',
       'TotalLiab_to_TangibleAssets_1', 'TotalLiab_to_TangibleAssets_cagr',
       'WC_to_TO_1', 'WC_to_TO_2_1', 'WC_to_TO_cagr', 'WORKING_CAPITAL_1']

df_vif_checked=x_train_final[final_vif_passed_list]
woe_data=pd.concat([df_vif_checked,y_train,X_train['Industry_Class']],axis=1)

print("# of rows in woe_data : %d" %len(woe_data.axes[0]))
print("# of columns in woe_data : %d" %len(woe_data.axes[1]))

# of rows in woe_data : 27276
# of columns in woe_data : 66


In [16]:
# WoE Transformation and IV calculation : 
# import pandas as pd
# import numpy as np
# import pandas.core.algorithms as algos
# from pandas import Series
# import scipy.stats.stats as stats
# import re
# import traceback
# import string


# define a binning function
def iv_woe(data, target,industry_class, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
   
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target,industry_class])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
#             binned_x = data.groupby('Industry_Class')[ivars].transform(lambda x : pd.qcut(x, bins, duplicates='drop',labels=None))

            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF



iv, woe = iv_woe(data = woe_data, target = 'Target',industry_class='Industry_Class', bins=10, show_woe = False)



Information value of CASH_AND_CASH_EQUIVALENTS_floored2 is 0.156437
Information value of CURRENT_LIABILITIES_2 is 0.032775
Information value of CashFlow_2_1 is 0.067455
Information value of CashFlow_2_1_to_Equity is 0.089427
Information value of CashFlow_2_1_to_Turnover is 0.070233
Information value of Cash_to_TO_1 is 0.1707
Information value of Cash_to_TO_cagr is 0.142353
Information value of Cash_to_Total_assets_1 is 0.138452
Information value of Cash_to_Total_assets_2_1 is 0.127995
Information value of Curr_to_Total_Assets_2_1 is 0.037253
Information value of Curr_to_Total_Assets_cagr is 0.013841
Information value of Current_Ratio_1 is 0.150853
Information value of Current_Ratio_2_1 is 0.018589
Information value of Current_Ratio_cagr is 0.018583
Information value of DEPRECIATION_2 is 0.022901
Information value of Debt_to_TNW_1 is 0.088891
Information value of EBITDA_to_Tangible_Assets_2 is 0.025735
Information value of EBITDA_to_Tangible_Assets_cagr is 0.100521
Information value of 

In [17]:
##Selecting IV value threshold at 0.03 and filtering out variables having higher IVs than this : 
iv_df=pd.DataFrame(woe.groupby('Variable')['IV'].sum()).reset_index()
final_iv_vars_list=iv_df[iv_df['IV']>=0.05].Variable.tolist()
len(final_iv_vars_list)

49

In [18]:

woe['Cutoff']=woe['Cutoff'].astype(str)

woe['Cutoff'] = woe['Cutoff'].str.replace('\[|\(|\]','')

woe[['MIN_VALUE','MAX_VALUE']] = woe['Cutoff'].str.split(',',expand=True)
woe['MIN_VALUE']=woe['MIN_VALUE'].astype(float)
woe['MAX_VALUE']=woe['MAX_VALUE'].astype(float)

woe = woe.rename(columns={'Variable': 'VAR_NAME', 'WoE': 'WOE'})
woe
final_iv=woe[['VAR_NAME', 'MIN_VALUE', 'MAX_VALUE','WOE']]
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,WOE
0,CASH_AND_CASH_EQUIVALENTS_floored2,-0.8650,-0.4110,0.340836
1,CASH_AND_CASH_EQUIVALENTS_floored2,-0.4110,-0.4000,0.069692
2,CASH_AND_CASH_EQUIVALENTS_floored2,-0.4000,-0.3680,0.191302
3,CASH_AND_CASH_EQUIVALENTS_floored2,-0.3680,-0.3110,0.068939
4,CASH_AND_CASH_EQUIVALENTS_floored2,-0.3110,-0.2130,0.301316
...,...,...,...,...
5,WORKING_CAPITAL_1,-0.2260,-0.0918,0.151571
6,WORKING_CAPITAL_1,-0.0918,0.1160,-0.374095
7,WORKING_CAPITAL_1,0.1160,0.4640,-0.192788
8,WORKING_CAPITAL_1,0.4640,1.1560,-0.310170


In [19]:
def WOE_VARIABLE_DF(final_iv,X):
    
    final_iv_c= final_iv[['VAR_NAME', 'MIN_VALUE', 'MAX_VALUE','WOE']]
    final_iv_list= pd.DataFrame(final_iv_c.groupby('VAR_NAME')['MIN_VALUE'].apply(list))
    final_iv_l= pd.DataFrame(final_iv_c.groupby('VAR_NAME')['WOE'].apply(list))
    final_iv_list['WOE']= final_iv_l['WOE']
    final_iv_list=final_iv_list.reset_index()
    variable_list= list(set(final_iv_c.VAR_NAME))
    X_WOE= pd.DataFrame()
    
    for var in variable_list:
        a=final_iv_list["MIN_VALUE"][final_iv_list["VAR_NAME"]==var].to_list()[0]
        aa=final_iv_list["WOE"][final_iv_list["VAR_NAME"]==var].to_list()[0]
        b= list(X[var])
        c1=list(np.digitize(b,a, right= True))
        c=[x-1 for x in c1]
        c2=[i if i != -1 else 0 for i in c]
        c3=[aa[i] for i in c2]
        X_WOE[var]=c3
    return X_WOE

In [20]:
## LEt's now create the WoE Transformed data set for both training and testing purposes : 

X_WOE_train= WOE_VARIABLE_DF(final_iv,X_train)
X_WOE_test= WOE_VARIABLE_DF(final_iv,X_test)

X_WOE_train

df_trn_ot_mvi_WoE= WOE_VARIABLE_DF(final_iv,df_trn_ot_mvi)
df_trn_ot_mvi_WoE

,RE_to_Equity_1,EBITDA_to_Tangible_Assets_2,TO_to_Assets_2_1,NET_INTANGIBLES_1,EBITDA_to_curr_liab_2,Current_Ratio_cagr,CASH_AND_CASH_EQUIVALENTS_floored2,NPBT_to_TNW_2_1,TotalLiab_to_TangibleAssets_1,NPBT_to_TNW_1,...,Cash_to_Total_assets_2_1,NET_INCOME_1,Cash_to_Total_assets_1,TO_to_Assets_cagr,TotalLiab_to_TNW_2_1,GROSS_PROFIT_1,Net_Income_Margin_cagr,RE_to_Tangible_Assets_2_1,NetFixedAssets_to_TotalDebt_1,High_Risk_Ind
0,-0.038395,0.024638,0.125631,-0.046365,-0.005215,-0.029908,-0.120712,-0.086625,0.039236,-0.331162,...,-0.491773,0.082925,-0.567387,0.022759,0.009823,0.009071,0.151949,0.178723,-0.211511,-0.507987
1,-0.171997,0.165229,0.193571,-0.049476,0.164852,-0.045056,0.069692,0.401064,-0.842135,-0.088125,...,0.009447,-0.155947,0.325056,0.068186,-0.563504,-0.249999,0.151949,-0.174021,-0.230577,-0.507987
2,-0.171997,0.082925,0.141142,-0.049476,-0.352589,-0.290340,0.191302,-0.086625,0.068186,-0.022361,...,0.390427,0.054378,-0.567387,-0.417427,0.037733,0.009447,-0.021610,-0.174021,-0.211511,-0.507987
3,-0.171997,0.151949,-0.039741,-0.046365,0.191680,-0.034858,0.068939,-0.086625,0.671371,0.124500,...,-0.419293,0.462771,0.164852,-0.085875,0.000442,0.009447,0.151949,0.126009,-0.230577,-0.507987
4,0.903475,0.216750,-0.193910,-0.046365,-0.138555,-0.029908,0.301316,0.401064,0.422408,0.112586,...,0.009447,0.462771,0.325056,0.165984,0.000442,0.165984,-0.492145,-0.396820,-0.230577,-0.507987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34090,-0.332654,-0.004839,-0.138930,-0.046365,0.082172,-0.034858,0.069692,-0.086625,0.124877,0.301316,...,0.009447,-0.913964,0.164852,-0.085875,-0.097966,0.253817,-0.420038,-0.396820,0.171285,-0.507987
34091,-0.171997,0.216750,0.193571,-0.046365,-0.005215,-0.045056,0.068939,0.401064,0.124877,-0.396820,...,0.447171,-0.155947,0.164852,0.165984,0.000442,-0.249999,0.462771,-0.397938,0.171285,-0.507987
34092,-0.332654,0.151949,0.141142,-0.046365,0.082172,-0.034858,-0.842505,-0.330789,-0.963857,-0.088125,...,-0.568503,0.054378,-0.007094,-0.091490,0.000442,0.204480,-0.492145,0.126009,-0.211137,-0.507987
34093,-0.332654,-0.004839,-0.138930,-0.046365,0.082172,-0.034858,0.069692,-0.086625,0.124877,0.301316,...,0.009447,-0.913964,0.164852,-0.085875,-0.097966,0.253817,-0.420038,-0.396820,0.171285,-0.507987


In [21]:
### Feature selection stage -4 :  

import warnings
warnings.filterwarnings('ignore') 

#Loop through for selecting features using RFE (Recursive Feature Extraction)

x_train_final_1=X_train[final_iv_vars_list].copy()

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression


#no of features
nof_list=np.arange(1,len(final_iv_vars_list))  
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    model = LogisticRegression(solver='liblinear',class_weight="balanced",max_iter=2000)
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(x_train_final_1,y_train)
    #X_test_rfe = rfe.transform(x_train_final_1)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_train_rfe,y_train)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

Optimum number of features: 37
Score with 37 features: 0.603681


In [23]:
#############************** Building the final RFE selection with 37 vars ***********************###################  :

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

x_train_final_1=X_train[final_iv_vars_list].copy()
cols=list(x_train_final_1.columns)
model = LogisticRegression(solver='liblinear',class_weight="balanced",max_iter=2000)
rfe = RFE(model, n_features_to_select=37)



# y_train= df_trn_ot_mvi['Target'].astype(int).values.reshape(-1, 1)

X_rfe_train = rfe.fit_transform(x_train_final_1,y_train)

#Fitting the data to model
model.fit(X_rfe_train,y_train) 




temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

Index(['CASH_AND_CASH_EQUIVALENTS_floored2', 'CashFlow_2_1',
       'CashFlow_2_1_to_Turnover', 'Cash_to_TO_1', 'Cash_to_Total_assets_1',
       'Current_Ratio_1', 'Debt_to_TNW_1', 'EBITDA_to_Tangible_Assets_cagr',
       'EBITDA_to_curr_liab_2', 'EBITDA_to_curr_liab_2_1', 'EBIT_to_TO_cagr',
       'Equity_to_Assets_cagr', 'Equity_to_Liability_2',
       'Equity_to_Liability_2_1', 'FLAG_LIMITED', 'High_Risk_Ind',
       'Low_Risk_Ind', 'NET_FIXED_ASSETS_2', 'NET_INCOME_1', 'NPBT_to_TNW_1',
       'NPBT_to_TNW_2_1', 'NetFixedAssets_to_TotalDebt_1',
       'Net_Income_Margin_1', 'Net_Income_Margin_2_1',
       'Net_Income_Margin_cagr', 'RETAINED_EARNINGS_1', 'RE_to_Equity_1',
       'RE_to_Tangible_Assets_cagr', 'ROCE_2_1',
       'TotalDebt_to_TangibleAssets_2', 'TotalLiab_to_TNW_2',
       'TotalLiab_to_TNW_2_1', 'TotalLiab_to_TNW_cagr',
       'TotalLiab_to_TangibleAssets_1', 'TotalLiab_to_TangibleAssets_cagr',
       'WC_to_TO_1', 'WORKING_CAPITAL_1'],
      dtype='object')


In [165]:
## Building the final Model with raw variables: 

final_features_dt=list(selected_features_rfe)



from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score ,recall_score
from sklearn.metrics import roc_auc_score ,roc_curve, auc

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')


def display_summary(true,pred):
    tn, fp, fn, tp = confusion_matrix(true,pred).ravel()
    print('confusion matrix')
    print(np.array([[tp,fp],[fn,tn]]))
    print('sensitivity is %f',1.*tp/(tp+fn))
    print('specificity is %f',1.*tn/(tn+fp))
    print('accuracy is %f',1.*(tp+tn)/(tp+tn+fp+fn))
    print('balanced accuracy is %',1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp)))
 



##CHECK CLASS PROPORTIONS IN THE DATA
s=y_train.value_counts()
m1=1- s[1]/(s[0]+s[1])
m2=s[1]/(s[0]+s[1])


model_tree = LogisticRegression(solver='liblinear',class_weight="balanced",max_iter=20000,C=10000000000,penalty='l2')
# model_tree = LogisticRegression(solver='liblinear',max_iter=2000,class_weight={0:m1,1:m2})
# model_tree = LogisticRegression(solver='liblinear',max_iter=2000,C=1000,penalty='l2')
# model_tree = DecisionTreeClassifier(random_state=42,class_weight="balanced")
# model_tree = RandomForestClassifier(random_state=0,class_weight="balanced",n_estimators=37,bootstrap=True,criterion='gini',max_features='auto')
# model_tree = DecisionTreeClassifier(random_state=0)

# model_tree= GaussianNB()





# model_tree= XGBClassifier(learning_rate =0.5,max_depth = 5,
#                           subsample=0.8,colsample_bytree=0.8,min_child_weight = 1,
#                           objective= 'binary:logistic',alpha=0,scale_pos_weight =2,gamma=0,
#                           max_delta_step =1)

# model_tree= XGBClassifier()
tree = model_tree.fit(X_train[final_features_dt],y_train)

ypred_train_dt = model_tree.predict(X_train[final_features_dt])
ypred_test_dt = model_tree.predict(X_test[final_features_dt])

Accu_Score_training =accuracy_score(y_train,ypred_train_dt)
Accu_Score_testing =accuracy_score(y_test,ypred_test_dt)

rec_score_training=recall_score(y_train,ypred_train_dt)
rec_score_testing=recall_score(y_test,ypred_test_dt)

auc_training = roc_auc_score(y_train,ypred_train_dt)
gini_training = (2*auc_training -1)

auc_testing = roc_auc_score(y_test,ypred_test_dt)
gini_testing = (2*auc_testing -1)



Actual_test_bad_rate=y_test.sum()/y_test.count()
predicted_test_bad_rate =ypred_test_dt.sum()/len(ypred_test_dt)

print("Accuracy_Score_Training_Raw : %f"  %Accu_Score_training)
print("Recall_Score_Training_Raw : %f"  %rec_score_training)
print("gini_Score_Training_Raw : %f"  %gini_training)
print("\t")
print("Accuracy_Score_Testing_Raw : %f"  %Accu_Score_testing)
print("Recall_Score_Testing_Raw : %f"  %rec_score_testing)
print("gini_Score_Testing_Raw : %f"  %gini_testing)


print("\t")

print("Percentage gini_Score_Training vs testing : %f"  %((gini_training-gini_testing)*100/gini_training))

print("\t")

print("Model stability on the testing data set : %f" %((predicted_test_bad_rate/Actual_test_bad_rate)-1))
print("\t")
print("Classification  Report : ")
print(classification_report(y_test, ypred_test_dt))
print("\t")
print("Confusion Matrix : ")
print(confusion_matrix(y_test, ypred_test_dt))
display_summary(y_test, ypred_test_dt)



Accuracy_Score_Training_Raw : 0.603571
Recall_Score_Training_Raw : 0.719880
gini_Score_Training_Raw : 0.320548
	
Accuracy_Score_Testing_Raw : 0.603314
Recall_Score_Testing_Raw : 0.708108
gini_Score_Testing_Raw : 0.308500
	
Percentage gini_Score_Training vs testing : 3.758671
	
Model stability on the testing data set : 14.037838
	
Classification  Report : 
              precision    recall  f1-score   support

           0       0.99      0.60      0.75      6634
           1       0.05      0.71      0.09       185

    accuracy                           0.60      6819
   macro avg       0.52      0.65      0.42      6819
weighted avg       0.96      0.60      0.73      6819

	
Confusion Matrix : 
[[3983 2651]
 [  54  131]]
confusion matrix
[[ 131 2651]
 [  54 3983]]
sensitivity is %f 0.7081081081081081
specificity is %f 0.600391920410009
accuracy is %f 0.6033142689543921
balanced accuracy is % 0.6542500142590586


In [169]:
##Predicting probability on training data set : 
import time
TodaysDate = time.strftime("%d/%m/%Y")
file_name=df_train_pred+TodaysDate
prob_train = model_tree.predict_proba(df_trn_ot_mvi[final_features_dt])
prob_train_df = pd.DataFrame(data=prob_train)
ypred_train_dt_full = model_tree.predict(df_trn_ot_mvi[final_features_dt])

##Concat new columns to original dataframe and write to csv : 
ypred_train_dt_df = pd.DataFrame(data=ypred_train_dt_full) 
df_train_pred = pd.concat([df_trn_ot_mvi[['Rec_ID','Target']], ypred_train_dt_df ,prob_train_df], axis=1)
df_train_pred.to_csv('C:/Users/45018883/Desktop/GRABBLE/df_train_pred.csv')

ValueError: unknown type str320

In [47]:
## Building the final Model with WoE variables: 

final_features_dt=list(selected_features_rfe)


from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score ,recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from numpy import loadtxt
from xgboost import XGBClassifier




# CHECK CLASS PROPORTIONS IN THE DATA
s=y_train.value_counts()
m1=1- s[1]/(s[0]+s[1])
m2=s[1]/(s[0]+s[1])
model_tree = LogisticRegression(solver='liblinear',class_weight="balanced",max_iter=20000,C=10000000000,penalty='l2')
# model_tree = LogisticRegression(solver='liblinear',class_weight={0:m1,1:m2},max_iter=20000,C=10000000000,penalty='l2')
# model_tree = LogisticRegression(solver='liblinear',class_weight="balanced",max_iter=2000)
# model_tree = LogisticRegression(solver='liblinear',max_iter=2000)
# model_tree = DecisionTreeClassifier(random_state=0,class_weight={0:m1,1:m2})
# model_tree = RandomForestClassifier(random_state=0)
# model_tree = DecisionTreeClassifier(random_state=0)
# model_tree= GaussianNB()
# # model_tree= XGBClassifier()
# model_tree= XGBClassifier(learning_rate =0.5,max_depth = 5,
#                           subsample=0.8,colsample_bytree=0.8,min_child_weight = 1,
#                           objective= 'binary:logistic',alpha=0,scale_pos_weight =2,gamma=0,
#                           max_delta_step =1)

tree = model_tree.fit(X_WOE_train[final_features_dt],y_train)

ypred_train_dt = model_tree.predict(X_WOE_train[final_features_dt])
ypred_test_dt = model_tree.predict(X_WOE_test[final_features_dt])

Accu_Score_training =accuracy_score(y_train,ypred_train_dt)
Accu_Score_testing =accuracy_score(y_test,ypred_test_dt)

rec_score_training=recall_score(y_train,ypred_train_dt)
rec_score_testing=recall_score(y_test,ypred_test_dt)

auc_training = roc_auc_score(y_train,ypred_train_dt)
gini_training = (2*auc_training -1)

auc_testing = roc_auc_score(y_test,ypred_test_dt)
gini_testing = (2*auc_testing -1)

Actual_test_bad_rate=y_test.sum()/y_test.count()
predicted_test_bad_rate =ypred_test_dt.sum()/len(ypred_test_dt)



print("Accuracy_Score_Training_WoE : %f"  %Accu_Score_training)
print("Recall_Score_Training_WoE : %f"  %rec_score_training)
print("gini_Score_Training_WoE : %f"  %gini_training)
print("\t")
print("Accuracy_Score_Testing_WoE : %f"  %Accu_Score_testing)
print("Recall_Score_Testing_WoE : %f"  %rec_score_testing)
print("gini_Score_Testing_WoE : %f"  %gini_testing)

print("\t")
print("Percentage gini_Score_Training vs testing : %f"  %((gini_training-gini_testing)*100/gini_training))

print("\t")

print("Model stability on the testing data set : %f" %((predicted_test_bad_rate/Actual_test_bad_rate)-1))
print("\t")

print("\t")
print("Confusion Matrix : ")
print(confusion_matrix(y_test, ypred_test_dt))

print(list(zip(tree.coef_, final_features_dt)))

# Predicting probability on training data set : 
prob_train = model_tree.predict_proba(df_trn_ot_mvi_WoE[final_features_dt])
prob_train_df = pd.DataFrame(data=prob_train)

# Concat new columns to original dataframe and write to csv : 
ypred_train_dt_full = model_tree.predict(df_trn_ot_mvi_WoE[final_features_dt])
ypred_train_dt_df = pd.DataFrame(data=ypred_train_dt_full) 
df_train_pred = pd.concat([df_trn_ot_mvi[['Rec_ID','Target']], ypred_train_dt_df ,prob_train_df], axis=1)
df_train_pred.to_csv('C:/Users/45018883/Desktop/GRABBLE/df_train_woe_pred.csv')

Accuracy_Score_Training_WoE : 0.652662
Recall_Score_Training_WoE : 0.709337
gini_Score_Training_WoE : 0.360585
	
Accuracy_Score_Testing_WoE : 0.642176
Recall_Score_Testing_WoE : 0.637838
gini_Score_Testing_WoE : 0.280135
	
Percentage gini_Score_Training vs testing : 22.310921
	
Model stability on the testing data set : 12.464865
	
	
Confusion Matrix : 
[[4261 2373]
 [  67  118]]
[(array([ 7.93444417e-01,  6.17301013e-01, -3.37806333e-02,  5.59005579e-02,
        5.27968941e-01,  1.35590361e-01, -8.86564109e-02,  4.98575917e-01,
        1.61030943e-01,  2.64315057e-01,  3.24181439e-01,  5.24690214e-01,
       -5.54954820e-02,  3.49366704e-02, -1.66810490e-04, -1.53309646e-04,
        4.80921283e-05,  8.21759529e-01,  3.14775160e-01, -2.36140193e-02,
        2.62086699e-01,  4.43196537e-01,  2.67150985e-01,  3.26851659e-01,
       -2.53203997e-01, -4.68339736e-01,  7.31754914e-01, -2.12563947e-01,
       -4.18308769e-03,  4.38541354e-01,  1.49889202e-01,  1.25808463e-01,
        6.323919

In [50]:
##Getting Feature Importanace out of the logtisc regression : 
parameters = pd.DataFrame(tree.coef_)
parameters.to_csv('C:/Users/45018883/Desktop/GRABBLE/feature_importances.csv')
final_features_df=pd.DataFrame(final_features_dt)
final_features_df.to_csv('C:/Users/45018883/Desktop/GRABBLE/final_features_df.csv')

In [54]:
##Reading and treating the test data set now :


pd.options.mode.chained_assignment = None 



df_test=pd.read_csv('C:/Users/45018883/Desktop/GRABBLE/GRAbble_Test.csv')



##Outlier Treatment of the selected features  : 
df_test.replace([-np.Inf,np.Inf], np.nan,inplace=True)
df_test_ot= df_test.groupby(df_test.Industry.values).transform(lambda x: np.clip(x,x.quantile(0.05),x.quantile(0.95))if x.name in num_list else x).copy()



# Missing  Value Treatment of the selected features : 
df_test_ot_mvi = df_test_ot.fillna(df_test_ot.groupby('Industry').transform('median'))
df_check_test=df_test_ot_mvi.isnull().sum()
df_check2_test=pd.DataFrame({'Variable':df_check_test.index, '#Missing_Values':df_check_test.values})
df_check2_test[df_check2_test['#Missing_Values']>0]



## Normalize the entire data set :

from sklearn.preprocessing import StandardScaler
df_test_ot_mvi[num_list] = StandardScaler().fit_transform(df_test_ot_mvi[num_list])
df_test_ot_mvi.head()

# x_test=df_test_ot_mvi[final_features_dt]

# x_test.head()

##Merge with the main data set : 
df_test_ot_mvi =df_test_ot_mvi.merge(df_industry_class,on=['Industry'],how='left')

##Create industry class dummies and merge to the original data set :
ind_dummies=pd.get_dummies(df_test_ot_mvi['Industry_Class'])
ind_dummies=ind_dummies.drop('Med_Risk_Ind',axis=1)

df_test_ot_mvi = pd.concat([df_test_ot_mvi, ind_dummies], axis=1)
df_test_ot_mvi.head()


numlist_ind_dummy=num_list+['High_Risk_Ind' ,'Low_Risk_Ind' ]
df_test_ot_mvi.head()






,Rec_ID,FLAG_NTB,FLAG_LIMITED,Industry,AUDITMETHOD_1,AUDITMETHOD_2,AUDITMETHOD_3,AUDITMETHOD_4,BAD_DEBT_RESERVE_1,BAD_DEBT_RESERVE_2,...,RE_to_Tangible_Assets_cagr,Cash_growth_2_1,CashFlow_2_1,CashFlow_cagr,CashFlow_2_1_to_Assets,CashFlow_2_1_to_Turnover,CashFlow_2_1_to_Equity,Industry_Class,High_Risk_Ind,Low_Risk_Ind
0,79722,-0.242463,0.530673,Services_ProfessionalSciTech,Compiled,NaN,NaN,NaN,78032.0,288298.0,...,-0.098840,-0.102920,-0.090491,-0.193663,-0.064702,-0.019727,-0.014253,High_Risk_Ind,1,0
1,79723,-0.242463,0.530673,Agriculture - Other,Compiled,Draft,Draft,Compiled,NaN,NaN,...,-0.214109,-0.101408,-0.078051,0.206746,-0.060436,0.064405,-0.014648,Low_Risk_Ind,0,1
2,79724,-0.242463,0.530673,Utilities,Audit Unqualif,Audit Unqualif,Audit Unqualif,Audit Unqualif,NaN,22591.5,...,0.469432,-0.080608,2.796830,1.227420,2.824400,5.371399,0.882275,High_Risk_Ind,1,0
3,79725,-0.242463,0.530673,Wholesale,Compiled,Compiled,Compiled,NaN,15531.5,10056.5,...,0.008580,-0.102933,-0.091463,-0.151309,-0.066125,-0.020834,-0.018596,High_Risk_Ind,1,0
4,79726,-0.242463,0.530673,Financial,Draft,Draft,Draft,Audit Unqualif,NaN,NaN,...,-0.131133,-0.102780,0.999324,-0.050612,0.552497,0.758499,0.245112,Low_Risk_Ind,0,1


In [63]:
df_test_grabble_WoE= WOE_VARIABLE_DF(final_iv,df_test_ot_mvi)
df_test_grabble_WoE.head()
x_test_grabble_WoE=df_test_grabble_WoE[final_features_dt]
x_test_grabble_WoE


,CASH_AND_CASH_EQUIVALENTS_floored2,CashFlow_2_1,CashFlow_2_1_to_Turnover,Cash_to_TO_1,Cash_to_Total_assets_1,Current_Ratio_1,Debt_to_TNW_1,EBITDA_to_Tangible_Assets_cagr,EBITDA_to_curr_liab_2,EBITDA_to_curr_liab_2_1,...,RE_to_Tangible_Assets_cagr,ROCE_2_1,TotalDebt_to_TangibleAssets_2,TotalLiab_to_TNW_2,TotalLiab_to_TNW_2_1,TotalLiab_to_TNW_cagr,TotalLiab_to_TangibleAssets_1,TotalLiab_to_TangibleAssets_cagr,WC_to_TO_1,WORKING_CAPITAL_1
0,0.340836,-0.230203,0.138877,0.362819,0.164852,-0.231325,-0.250373,-0.023862,-0.352589,-0.155947,...,-0.157070,0.054378,0.149474,-0.230203,-0.563504,-0.005215,-0.842135,0.080293,-0.541784,-0.310170
1,0.340836,-0.230203,0.138877,0.311354,0.216265,0.024262,0.251549,-0.121086,0.422408,0.471960,...,-0.421156,-0.420038,0.178346,0.190925,0.000442,-0.171022,0.039236,0.055507,-0.316869,0.009071
2,-0.120712,-0.467303,-0.680101,-0.877967,-0.007094,-1.216290,-0.545127,0.068562,-0.842505,-0.174021,...,0.097472,-0.467676,-0.443401,0.357871,0.009823,-0.054364,-0.963857,0.080293,-1.033598,-0.420038
3,0.340836,-0.230203,0.138877,0.304459,0.216265,-0.070365,0.521150,0.139631,0.191680,-0.086250,...,-0.103155,0.492353,0.191680,0.410836,0.266648,0.567896,0.124877,-0.398683,0.223196,0.178346
4,-0.842505,-0.466931,-0.492889,-0.877967,-1.032488,-0.231325,-0.267547,-0.249999,-0.352589,-0.331536,...,-0.210014,-0.230577,-0.249999,-0.568503,-0.563504,-0.155947,-0.481674,0.178723,-1.033598,-0.420038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10667,0.068939,0.069321,0.023886,-0.877967,0.325056,-0.443401,-0.267547,0.390427,-0.374468,-0.086250,...,-0.157070,0.492353,-0.269417,-0.992406,-0.381901,-0.005215,-0.963857,-0.307927,-1.033598,0.111454
10668,0.069692,0.024262,0.138877,0.311354,0.164852,0.097472,-0.267547,-0.121086,-0.374468,-0.331536,...,-0.121086,-0.230577,-0.269417,-0.568503,-0.381901,-0.171022,-0.842135,-0.397193,-0.193910,-0.374095
10669,-0.211137,0.096342,0.039236,0.311354,0.325056,-0.443401,-0.267547,-0.023862,-0.352589,-0.155947,...,-0.103155,0.054378,-0.249999,-0.652228,-0.174395,-0.157070,-0.842135,-0.398683,-0.006718,-0.420038
10670,-0.120712,-0.466931,-0.082114,-0.054364,-0.007094,0.358251,0.251549,0.390427,-0.374468,-0.086250,...,-0.037866,0.492353,0.149474,0.190925,0.482963,0.567896,0.039236,0.178723,0.299424,0.178346


In [64]:


#Predicting on test data
y_test_pred_grabble = model_tree.predict(x_test_grabble_WoE)
y_test_pred_df = pd.DataFrame(data=y_test_pred_grabble) 

#Scoring the test dataset
t = model_tree.predict_proba(x_test_grabble_WoE)
t_df = pd.DataFrame(data=t)

y_test_pred_df.iloc[:,0].value_counts()


0    7470
1    3202
Name: 0, dtype: int64

In [65]:
#MErging with the test data set : 

#Concat new columns to original dataframe 
df_test_pred = pd.concat([df_test, y_test_pred_df,t_df], axis=1)

df_test_pred.to_csv('C:/Users/45018883/Desktop/GRABBLE/df_test_pred.csv')
